In [ ]:
!pip install playwright

In [ ]:
from playwright.async_api import async_playwright, TimeoutError
import asyncio
import random
import shutil

async def xy_for_element(page, locator_arg):
    locator = page.locator(locator_arg)
    # wait for it?
    try:
        await locator.wait_for(state='visible', timeout=1000)
    except TimeoutError:
        raise TimeoutError(f"Could not find element {locator_arg}")
    bounding_box = await locator.bounding_box()
    # sample a 2D normal distribution point centered at the center of bounding box
    # with a standard deviation of 1/2 the width in x, and 1/2 height of the bounding box
    # and resample if the point is outside the bounding box

    res = -1, -1
    while not (0 <= res[0] <= bounding_box['width'] and 0 <= res[1] <= bounding_box['height']):
        res = (
            random.gauss(bounding_box['width'] / 2, bounding_box['width'] / 4),
            random.gauss(bounding_box['height'] / 2, bounding_box['height'] / 4)
        )
    return bounding_box['x'] + res[0], bounding_box['y'] + res[1]

async def moveto_and_click(page, selector, steps=20, sleep=0.5):
    target = await xy_for_element(page, selector)
    await page.mouse.move(*target, steps=steps)
    # quickly check for target if it moved
    bb = await page.locator(selector).bounding_box()
    if not (bb['x'] <= target[0] <= bb['x'] + bb['width'] and bb['y'] <= target[1] <= bb['y'] + bb['height']):
        target = await xy_for_element(page, selector)
        await page.mouse.move(*target, steps=2)
    await page.mouse.click(*target)
    await asyncio.sleep(sleep)

async def prompt(page, prompt):
    await moveto_and_click(page, "#chat_message_box")
    await page.keyboard.type(f"{prompt}\n", delay=50)
    while True:
        await asyncio.sleep(1)
        if not await page.locator('#submit_button').is_hidden():
            break

async def upload_file(page, file, sleep=0.5):
    async with page.expect_file_chooser() as fc_info:
        await moveto_and_click(page, "#upload_button")

    file_chooser = await fc_info.value
    await file_chooser.set_files(file)
    await asyncio.sleep(sleep)

async def run_demo(procedure, save_video_as=None):
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=False,
            slow_mo=50
        )
        context = await browser.new_context(
            color_scheme="dark",
            geolocation={"longitude": 12.492507, "latitude": 41.889938},
            record_video_dir="videos/",
            record_video_size={"width": 1920, "height": 1080},
            viewport={"width": 1280, "height": 720},
        )
        page = await context.new_page()
        await page.add_init_script(
            path="mouse_helper.js"
        )
        await procedure(page)
        video_path = await page.video.path()
        await context.close()
        await browser.close()
    if save_video_as is not None:
        shutil.move(video_path, save_video_as)

In [ ]:
async def demo1(page):
    await page.goto("localhost:7860")

    try:
        # a way to wait for page to be ready, is waiting for the load_model button to appear
        await page.wait_for_selector("#load_model_button", timeout=5000)
    except TimeoutError:
        pass

    await moveto_and_click(page, "#model_selection_dropdown")
    await moveto_and_click(page, "text=gpt-4")
    try:
        await moveto_and_click(page, "#load_model_button")
    except TimeoutError:
        pass

    await upload_file(page, "data/world_happiness_report.csv")

    await prompt(page, "Who is happier, US or Canada?")
    await prompt(page, "Can you plot a scatter plot of happiness vs. gdp, and change the size of the dots based on life expectancy?")
    await asyncio.sleep(3) # let the graphic sink in

    await upload_file(page, "data/country-regions.csv")
    
    await prompt(page, "Can we color each dot in the previous plot based on the region?")
    await asyncio.sleep(3) # let the graphic sink in

    await prompt(page, "Can we add a line of best fit and show the score for correlation?")
    await asyncio.sleep(6)

In [ ]:
await run_demo(demo1, save_video_as="demo1.webm")

In [ ]:
# # Use FFMPEG to clean up the video
# ffmpeg -i demos/demo1.webm demo1.mp4
# ffmpeg -i demo1.mp4 -ss 0.5 -c:v libx264 -c:a copy -crf 23 -preset fast demo_cleaned.mp4
# ffmpeg -sseof -1 -i demo_cleaned.mp4 -update 1 -q:v 1 last_frame.jpg
# ffmpeg -loop 1 -i last_frame.jpg -c:v libx264 -t 1 -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" 1_frame.mp4
# echo -e "file '1_frame.mp4'\nfile 'demo_cleaned.mp4'" > concat_list.txt
# ffmpeg -f concat -safe 0 -i concat_list.txt -c copy final_demo.mp4
# rm last_frame.jpg 1_frame.mp4 concat_list.txt
# mv final_demo.mp4 datadm_happiness_qa_and_plots.mp4